
# Forum des métiers - Répartition de groupes et allocation de slots

Ce notebook permet d'effectuer des allocations individuelles à des slots de visite de représentants des métiers

Contraintes :

- pas d'élève sans affectation de métier pour un créneau (timeslot) donné,
- le moins possible de métier (job) sans affectation de groupe pour un timeslot donné,
- nombre de souhaits par élève =< timeslots,
- il peut y avoir n représentants pour un métier donné.

Le choix d'affectation utilisé donne la priorité au remplissage de tous les slots des intervenants de manière équilibrée.

La note globale de satisfaction du choix des élèves est un peu plus élevée que sur une stratégie d'affectation selon les préférences élèves pures et le taux d'intervenants sans élève est le plus faible.

La conversion du contenu d'un fichier Excel au formal JSON peut être effectuée sur le site suivant :
https://tableconvert.com/excel-to-json

## Importations des modules

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import math
import random
from json import loads, dumps
from numpy.random import default_rng
from dataclasses import dataclass
from collections import defaultdict

In [2]:
from modules.typedef import *
from modules.q_utils import *
from modules.student_utils import get_most_interested_student_for_job, get_copy_students, convert_to_student_timetable
from modules.timetable_utils import timetable_student_view, get_timetable_intervenants
from modules.evaluation import get_evaluation

# Allocations des élèves aux timeslots

Un timeslot est une liste des intervenants de métiers pour lesquels la file d'attente 'Q' est renseignée avec les identifiants des élèves qui visiteront cet intervenant pour ce timeslot.

Timeslot :

```Python
[ { 'id' : int , 'batch_size' : int, 'Q' : [ int ] } ]
```

Timetable :

```Python
[ Timeslot ]
```

## Q-intervenant

Pour chaque intervenant considéré tour à tour (et dont le métier ne figure pas dans la liste des métiers à ne plus considérer), par ordre d'intervenant dont le taux de remplissage de Q va du plus faible au plus important, on cherche à déterminer un élève ayant un choix (de rang de préférence le plus élevé) correspondant à la catégorie de métier représenté par cet intervenant. 

Cet élève ne doit pas avoir visité cet intervenant auparavant.

Si un élève est trouvé, on retire l'élève de la liste des candidats et on ajoute cet élève à la Q de l'intervenant.

Si aucun élève n'est trouvé, on ajoute le métier représenté par l'intervenant à la liste des métiers à ne plus considérer (blacklist).

## Queing intervenants load balance driven

In [30]:
def q_students_itv_driven_pass1(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student])->Tuple[Q_Intervenants, list[Student], list[Student]]:
    NB_LOOP_MAX = 1000

    result_students = get_copy_students(students)
    remaining_students = result_students
    
    no_more_Q = False
    blacklisted_jobs=[]
    nb_loops = 0

    while((len(remaining_students)>0) and (not no_more_Q)):

        assert(nb_loops < NB_LOOP_MAX)
        nb_loops += 1

        # liste des intervenants ayant une file de visite Q qui n'est pas encore remplie et
        # classée du taux de remplissage de Q le plus faible vers le plus élevé.
        id_intervenants = [ id_itv for id_itv in get_next_q_to_load_sorted(q_intervenants) ]

        # liste [(job, intervenant)] pour chaque intervenant identifié précédement
        l_job_itv = [ get_job_itv_from_itv_id(q_intervenants, id_intervenant) for id_intervenant in id_intervenants  ]

        # list [(job, intervenant)] pour les jobs qui ne sont pas blacklisté i.e
        # pour les jobs pour lesquels il existe encore des élèves qui souhaitent visiter ce job
        l_job_itv_no_blacklisted = [ (job,itv) for (job,itv) in l_job_itv if not job in blacklisted_jobs ]

        if (len(l_job_itv_no_blacklisted)==0):
            # Il n'y a plus de job a traiter. Soit toutes les Q d'intervenants sont remplies soit 
            # tous les jobs sont maintenant blacklistés (i.e. aucun élève trouvé ayant un souhait pour tous ces 
            # métiers)

            # exit while loop
            no_more_Q = True
        else:

            # le job avec l'intervenant ayant sa file d'attente la moins remplie
            # est en tête de liste avec :
            # head is (job,itv), tail is *_ unused
            (job, itv), *_ = l_job_itv_no_blacklisted
            
            # chercher l'élève qui a émis le souhait de rang le plus élevé pour 
            # le job identifié comme ayant un intervenant avec la file d'attente la moins
            # remplie
            #     
            most_interested_student = get_most_interested_student_for_job(remaining_students,job)
            if most_interested_student != None:

                # ajouter (job, itv) dans la liste des itv_visited (slots) de l'élève 
                most_interested_student['itv_visited'].append((n_slot, (job,itv)))

                # ajouter l'élève dans la Q de l'intervenant pour le job
                q_intervenants[job][itv]['Q'].append(most_interested_student['id'])

                # remove the most interested student from remaining students
                remaining_students = [ s for s in remaining_students if s['id'] != most_interested_student['id']]
            else:
                # aucun élève intéressé par le job en pass1, ajouter le job en blacklist
                blacklisted_jobs.append(job)

    return q_intervenants, result_students, remaining_students

In [20]:
def q_students_itv_driven_pass2(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student], remaining_students:list[Student])->Tuple[Q_Intervenants, list[Student]]:
    result_students = get_copy_students(students)

    for student in remaining_students:
        
        # liste des intervenants ayant une file de visite Q qui n'est pas encore remplie et
        # classée du taux de remplissage de Q le plus faible vers le plus élevé.

        id_intervenants = [ id_itv for id_itv in get_next_q_to_load_sorted(q_intervenants) ]
        
        # liste [(job, intervenant)] pour chaque intervenant identifié précédement
        l_q_job_itv = [ get_job_itv_from_itv_id(q_intervenants, id_intervenant) for id_intervenant in id_intervenants  ]

        job_yet_visited = { j for (s,(j,_)) in student['itv_visited'] }
        job_itv_yet_visited = { (j,i) for (s,(j,i)) in student['itv_visited'] }

        # liste des (job, itv) pour lesquels job n'a pas encore été visité par l'élève
        next_q_job_itv_1 = [ (j,i) for (j,i) in l_q_job_itv if not j in job_yet_visited ]

        # liste des (job, itv) pour lesquels (job,itv) n'a pas encore été visité
        next_q_job_itv_2 = [ (j,i) for (j,i) in l_q_job_itv if not (j,i) in job_itv_yet_visited ]

        job, itv = None, None

        if (len(next_q_job_itv_1)==0):
            if (len(next_q_job_itv_2)==0):
                print("(!) free student")
            else:
                # solution par défaut: on refait visiter un même job mais pas le même intervenant
                job, itv = next_q_job_itv_2[0]
        else:
            # solution préférentielle : on fait visiter un nouveau job
            job, itv = next_q_job_itv_1[0]

        if job != None:
            r_student = [ s for s in result_students if s['id'] == student['id'] ][0]
            r_student['itv_visited'].append((n_slot, (job, itv)))
            
            #q_intervenants = push_student_in_q(q_intervenants, job, itv, student['id'])
            q_intervenants[job][itv]['Q'].append(student['id'])


    return q_intervenants, result_students



In [ ]:
def get_timetable_itv_driven(nb_slots:int, students:list[Student], intervenants_data:pd.DataFrame)->Tuple[Timetable, list[Student]]:
    timetable = []
    for n_slot in range(nb_slots):
        #print("MAKING SLOT {}".format(n_slot))

        q_intervenants = init_q_intervenants(intervenants_data)
        q_intervenants, students, remaining_students = q_students_itv_driven_pass1(n_slot, q_intervenants, students)
        
        #print("after pass 1 - students {}".format(students))
        
        q_intervenants, students = q_students_itv_driven_pass2(n_slot, q_intervenants, students, remaining_students)
        timetable.append(q_intervenants)
    return timetable, students

## Tests

### Test 1 : q_students_itv_driven - allocation des élèves pour un slot

In [31]:
# test : q_students_itv_driven
test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]

test_q_intervenants = init_q_intervenants(test_intervenants_data)
test_q_intervenants, test_students, remaining_students = q_students_itv_driven_pass1(0, test_q_intervenants, test_students)
test_q_intervenants, test_students = q_students_itv_driven_pass2(0, test_q_intervenants, test_students, remaining_students)

#
# Expected result :
#

expected_result = ([[],
  [{'id': 10, 'batch_size': 3, 'Q': [3107, 3109, 3111]},
   {'id': 11, 'batch_size': 2, 'Q': [3108, 3110]}],
  [{'id': 20, 'batch_size': 3, 'Q': [3100, 3103, 3105]},
   {'id': 21, 'batch_size': 2, 'Q': [3101, 3104]}],
  [{'id': 30, 'batch_size': 2, 'Q': [3112, 3106]}],
  [{'id': 40, 'batch_size': 1, 'Q': [3102]}],
  [{'id': 50, 'batch_size': 1, 'Q': []}]],
 [{'id': 3100, 'wishes': [2, 1, 3], 'itv_visited': [(0, (2, 0))]},
  {'id': 3101, 'wishes': [2, 1, 3], 'itv_visited': [(0, (2, 1))]},
  {'id': 3102, 'wishes': [2, 1, 4], 'itv_visited': [(0, (4, 0))]},
  {'id': 3103, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 0))]},
  {'id': 3104, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 1))]},
  {'id': 3105, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 0))]},
  {'id': 3106, 'wishes': [2, 1, 3], 'itv_visited': [(0, (3, 0))]},
  {'id': 3107, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 0))]},
  {'id': 3108, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3109, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 0))]},
  {'id': 3110, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3111, 'wishes': [1, 2, 4], 'itv_visited': [(0, (1, 0))]},
  {'id': 3112, 'wishes': [1, 3, 4], 'itv_visited': [(0, (3, 0))]}])

(test_q_intervenants, test_students) == expected_result

True

In [7]:
# test : get_timetable_itv_drivent

test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]


NB_SLOTS = 3
_, test_students = get_timetable_itv_driven(NB_SLOTS, test_students, test_intervenants_data)
timetable = convert_to_student_timetable(NB_SLOTS, test_students)


In [8]:
timetable

[[(3100, (0, (2, 0))),
  (3101, (0, (2, 1))),
  (3102, (0, (4, 0))),
  (3103, (0, (2, 0))),
  (3104, (0, (2, 1))),
  (3105, (0, (2, 0))),
  (3106, (0, (3, 0))),
  (3107, (0, (1, 0))),
  (3108, (0, (1, 1))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 1))),
  (3111, (0, (1, 0))),
  (3112, (0, (3, 0)))],
 [(3100, (1, (1, 1))),
  (3101, (1, (3, 0))),
  (3102, (1, (2, 0))),
  (3103, (1, (4, 0))),
  (3104, (1, (1, 0))),
  (3105, (1, (1, 1))),
  (3106, (1, (2, 1))),
  (3107, (1, (2, 0))),
  (3108, (1, (2, 1))),
  (3109, (1, (3, 0))),
  (3110, (1, (2, 0))),
  (3111, (1, (5, 0))),
  (3112, (1, (1, 0)))],
 [(3100, (2, (3, 0))),
  (3101, (2, (1, 0))),
  (3102, (2, (1, 1))),
  (3103, (2, (1, 0))),
  (3104, (2, (4, 0))),
  (3105, (2, (5, 0))),
  (3106, (2, (1, 1))),
  (3107, (2, (3, 0))),
  (3108, (2, (2, 0))),
  (3109, (2, (2, 0))),
  (3110, (2, (2, 1))),
  (3111, (2, (2, 1))),
  (3112, (2, (2, 0)))]]

In [9]:
#pd.DataFrame(get_timetable_intervenants(timetable, test_intervenants_data))
test_q_intervenants = init_q_intervenants(test_intervenants_data)
get_timetable_intervenants(timetable, test_q_intervenants)

nb_slots : [0, 1, 2]


[{10: 3, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 0},
 {10: 2, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 1},
 {10: 2, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 1}]

In [10]:
get_evaluation(timetable, test_students, test_q_intervenants)

overall_student_satisfaction : 0.0
overall ratio visited 1.0
nb_slots : [0, 1, 2]
nb_no_visits_itv : 1


In [11]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['2-0', '1-1', '3-0'], 3101: ['2-1', '3-0', '1-0'], 3102: ['4-0', '2-0', '1-1'], 3103: ['2-0', '4-0', '1-0'], 3104: ['2-1', '1-0', '4-0'], 3105: ['2-0', '1-1', '5-0'], 3106: ['3-0', '2-1', '1-1'], 3107: ['1-0', '2-0', '3-0'], 3108: ['1-1', '2-1', '2-0'], 3109: ['1-0', '3-0', '2-0'], 3110: ['1-1', '2-0', '2-1'], 3111: ['1-0', '5-0', '2-1'], 3112: ['3-0', '1-0', '2-0']})


defaultdict(list,
            {3100: ['2-0', '1-1', '3-0'],
             3101: ['2-1', '3-0', '1-0'],
             3102: ['4-0', '2-0', '1-1'],
             3103: ['2-0', '4-0', '1-0'],
             3104: ['2-1', '1-0', '4-0'],
             3105: ['2-0', '1-1', '5-0'],
             3106: ['3-0', '2-1', '1-1'],
             3107: ['1-0', '2-0', '3-0'],
             3108: ['1-1', '2-1', '2-0'],
             3109: ['1-0', '3-0', '2-0'],
             3110: ['1-1', '2-0', '2-1'],
             3111: ['1-0', '5-0', '2-1'],
             3112: ['3-0', '1-0', '2-0']})

## Tests données réelles
### Intervenants drivent

In [15]:
test_intervenants_data = pd.read_json('./data/2025/intervenants_data.json')

# importation d'un échantillon de données pour les tests unitaires
students_2025 = pd.read_json('./data/2025/data-2025.json')
result = students_2025.to_json(orient="index")
parsed = loads(result)
test_students_2025 = list(parsed.values())

In [16]:

NB_SLOTS = 6
timetable, test_students_2025 = get_timetable_itv_driven(NB_SLOTS, test_students_2025, test_intervenants_data)
#timetable


In [17]:

timetable = convert_to_student_timetable(NB_SLOTS, test_students_2025)
#timetable

In [18]:
q_intervenants = init_q_intervenants(test_intervenants_data)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))
#get_timetable_intervenants(timetable, test_intervenants_data)

nb_slots : [0, 1, 2, 3, 4, 5]


,10,11,12,20,21,30,31,40,50,51,52,53,54,55,60,61,62,63,70,80,81,82,83,90,91,92,93,94,95,100,101,102,110,111,112,113,114,120,130,131,140,141,142,150,151,160,161,162,163,170,171,172,180,181,182,190,191,192,200,210,211
0,4,4,4,3,3,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
1,4,4,4,4,4,4,4,4,4,4,4,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,3,4,4,4,4,4
2,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,3,3,4,4,4,4
3,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,3,4,4,3,4,4,4,4,4,4,4,4,4,4,3,4,4,4,4,3,4,4,4,4,4
4,4,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,4,4,3,3,4,4,4,4
5,4,4,4,4,4,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,3,3,4,4,4,4


In [19]:
get_evaluation(timetable, test_students_2025, q_intervenants)

overall_student_satisfaction : 0.0
overall ratio visited 1.2
nb_slots : [0, 1, 2, 3, 4, 5]
nb_no_visits_itv : 0


In [20]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['8-3', '3-0', '9-2', '6-1', '5-3', '7-0'], 3101: ['3-0', '19-1', '11-3', '7-0', '6-3', '21-1'], 3102: ['9-5', '16-2', '10-2', '11-0', '6-0', '15-0'], 3103: ['17-2', '16-3', '9-0', '8-3', '11-1', '7-0'], 3104: ['10-1', '11-3', '16-3', '17-0', '12-0', '13-0'], 3105: ['6-3', '11-0', '10-1', '3-0', '16-0', '12-0'], 3106: ['9-4', '16-0', '10-2', '8-0', '5-1', '11-1'], 3107: ['9-2', '16-0', '20-0', '11-1', '6-2', '15-1'], 3108: ['11-3', '13-1', '3-1', '10-1', '5-2', '16-2'], 3109: ['1-0', '2-1', '3-1', '5-1', '11-0', '4-0'], 3110: ['1-1', '18-2', '9-1', '4-0', '16-1', '2-0'], 3111: ['10-0', '5-5', '18-0', '9-3', '8-2', '17-2'], 3112: ['8-2', '5-1', '6-3', '9-2', '4-0', '11-0'], 3113: ['7-0', '20-0', '3-1', '6-0', '8-1', '18-1'], 3114: ['14-2', '9-4', '21-1', '2-0', '8-0', '18-0'], 3115: ['4-0', '5-4', '16-3', '9-4', '17-0', '11-2'], 3116: ['8-0', '16-0', '10-0', '9-2', '6-1', '5-5'], 3117: ['5-5', '6-2', '9-1', '17-1', '14-0', '20-0'], 3118: ['13-1', '21-1

defaultdict(list,
            {3100: ['8-3', '3-0', '9-2', '6-1', '5-3', '7-0'],
             3101: ['3-0', '19-1', '11-3', '7-0', '6-3', '21-1'],
             3102: ['9-5', '16-2', '10-2', '11-0', '6-0', '15-0'],
             3103: ['17-2', '16-3', '9-0', '8-3', '11-1', '7-0'],
             3104: ['10-1', '11-3', '16-3', '17-0', '12-0', '13-0'],
             3105: ['6-3', '11-0', '10-1', '3-0', '16-0', '12-0'],
             3106: ['9-4', '16-0', '10-2', '8-0', '5-1', '11-1'],
             3107: ['9-2', '16-0', '20-0', '11-1', '6-2', '15-1'],
             3108: ['11-3', '13-1', '3-1', '10-1', '5-2', '16-2'],
             3109: ['1-0', '2-1', '3-1', '5-1', '11-0', '4-0'],
             3110: ['1-1', '18-2', '9-1', '4-0', '16-1', '2-0'],
             3111: ['10-0', '5-5', '18-0', '9-3', '8-2', '17-2'],
             3112: ['8-2', '5-1', '6-3', '9-2', '4-0', '11-0'],
             3113: ['7-0', '20-0', '3-1', '6-0', '8-1', '18-1'],
             3114: ['14-2', '9-4', '21-1', '2-0', '8-0', '18

In [21]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students

defaultdict(<class 'list'>, {3100: ['8-3', '3-0', '9-2', '6-1', '5-3', '7-0'], 3101: ['3-0', '19-1', '11-3', '7-0', '6-3', '21-1'], 3102: ['9-5', '16-2', '10-2', '11-0', '6-0', '15-0'], 3103: ['17-2', '16-3', '9-0', '8-3', '11-1', '7-0'], 3104: ['10-1', '11-3', '16-3', '17-0', '12-0', '13-0'], 3105: ['6-3', '11-0', '10-1', '3-0', '16-0', '12-0'], 3106: ['9-4', '16-0', '10-2', '8-0', '5-1', '11-1'], 3107: ['9-2', '16-0', '20-0', '11-1', '6-2', '15-1'], 3108: ['11-3', '13-1', '3-1', '10-1', '5-2', '16-2'], 3109: ['1-0', '2-1', '3-1', '5-1', '11-0', '4-0'], 3110: ['1-1', '18-2', '9-1', '4-0', '16-1', '2-0'], 3111: ['10-0', '5-5', '18-0', '9-3', '8-2', '17-2'], 3112: ['8-2', '5-1', '6-3', '9-2', '4-0', '11-0'], 3113: ['7-0', '20-0', '3-1', '6-0', '8-1', '18-1'], 3114: ['14-2', '9-4', '21-1', '2-0', '8-0', '18-0'], 3115: ['4-0', '5-4', '16-3', '9-4', '17-0', '11-2'], 3116: ['8-0', '16-0', '10-0', '9-2', '6-1', '5-5'], 3117: ['5-5', '6-2', '9-1', '17-1', '14-0', '20-0'], 3118: ['13-1', '21-1

,0,1,2,3,4,5
3100,8-3,3-0,9-2,6-1,5-3,7-0
3101,3-0,19-1,11-3,7-0,6-3,21-1
3102,9-5,16-2,10-2,11-0,6-0,15-0
3103,17-2,16-3,9-0,8-3,11-1,7-0
3104,10-1,11-3,16-3,17-0,12-0,13-0
3105,6-3,11-0,10-1,3-0,16-0,12-0
3106,9-4,16-0,10-2,8-0,5-1,11-1
3107,9-2,16-0,20-0,11-1,6-2,15-1
3108,11-3,13-1,3-1,10-1,5-2,16-2
3109,1-0,2-1,3-1,5-1,11-0,4-0


In [22]:
timetable_students.to_csv('output_timetable_students_itv_drivent.csv', sep=';', header=False)